In [2]:
!pip install sentence-transformers scikit-learn



   ---------------------------------------- 0.0/204.1 MB ? eta -:--:--
   ---------------------------------------- 2.4/204.1 MB 11.2 MB/s eta 0:00:19
    --------------------------------------- 5.0/204.1 MB 11.6 MB/s eta 0:00:18
   - -------------------------------------- 7.3/204.1 MB 11.6 MB/s eta 0:00:17
   - -------------------------------------- 10.0/204.1 MB 11.5 MB/s eta 0:00:17
   -- ------------------------------------- 12.3/204.1 MB 11.5 MB/s eta 0:00:17
   -- ------------------------------------- 14.9/204.1 MB 11.6 MB/s eta 0:00:17
   --- ------------------------------------ 17.3/204.1 MB 11.5 MB/s eta 0:00:17
   --- ------------------------------------ 19.9/204.1 MB 11.5 MB/s eta 0:00:16
   ---- ----------------------------------- 22.5/204.1 MB 11.6 MB/s eta 0:00:16
   ---- ----------------------------------- 24.9/204.1 MB 11.6 MB/s eta 0:00:16
   ----- ---------------------------------- 27.5/204.1 MB 11.6 MB/s eta 0:00:16
   ----- ---------------------------------- 29.9/204

In [15]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Load the scraped data (material_name and price)
scraped_df = pd.read_csv("incision/material_data.csv")

# Load the target CSV file (with missing prices)
target_df = pd.read_csv("incision/Materials in Protocols - Salford Royal Hospital (1).csv")


# Load the NLP model for embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Convert material names to lowercase and replace NaN values with an empty string
scraped_df["material_name"] = scraped_df["material_name"].astype(str).str.lower().str.strip()
target_df["material_name"] = target_df["material_name"].astype(str).str.lower().str.strip()

# Encode material names into vector embeddings
scraped_embeddings = model.encode(scraped_df["material_name"].tolist())
target_embeddings = model.encode(target_df["material_name"].tolist())

# Function to find the best match based on cosine similarity
def get_best_match_index(target_embedding, scraped_embeddings):
    similarities = cosine_similarity([target_embedding], scraped_embeddings)
    best_match_idx = similarities.argmax()
    best_match_score = similarities[0, best_match_idx]
    return best_match_idx if best_match_score > 0.7 else None  # Set a similarity threshold

# Find matches and fill in missing prices
matched_prices = []
for target_embedding in target_embeddings:
    best_match_idx = get_best_match_index(target_embedding, scraped_embeddings)
    if best_match_idx is not None:
        matched_prices.append(scraped_df.iloc[best_match_idx]["price"])
    else:
        matched_prices.append(None)  # No suitable match found

# Update target DataFrame with matched prices
target_df["price"] = matched_prices

# Save the updated target data
target_df.to_csv("updated_target_data.csv", index=False)

print("Prices updated using NLP-based matching!")


Prices updated using NLP-based matching!
